In [ ]:
import numpy as np
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

In [ ]:
class ChatBot:
    def __init__(self):

        link="microsoft/DialoGPT-large"
        self.chat_history_ids = None
        self.bot_input_ids = None
        self.end_chat = False
        self.tokenizer = AutoTokenizer.from_pretrained(link)
        self.model = AutoModelForCausalLM.from_pretrained(link)
        self.greet_user()
    def greet_user(self):

        print("ChatBot : Welcome, I am a ChatBot, ready to assist you.\n Type 'bye', 'quit', or 'exit' to end the chat\n")

    def user_input(self):
        text = input("User    : ")
        if text.lower().strip() in ['bye', 'quit', 'exit']:
            self.end_chat = True
            print('Quitting ChatBot')
        else:
            self.new_user_input_ids = self.tokenizer.encode(text + self.tokenizer.eos_token, return_tensors='pt')

    def generate_response(self):
        if self.chat_history_ids is not None:
            self.bot_input_ids = torch.cat([self.chat_history_ids, self.new_user_input_ids], dim=-1)
        else:
            self.bot_input_ids = self.new_user_input_ids

        self.chat_history_ids = self.model.generate(
            self.bot_input_ids, max_length=1000, pad_token_id=self.tokenizer.eos_token_id
        )

        response = self.tokenizer.decode(
            self.chat_history_ids[:, self.bot_input_ids.shape[-1]:][0], skip_special_tokens=True
        )

        if not response:
            response = self.get_fallback_response()

        print('ChatBot :', response)
    def get_fallback_response(self):
        i = -1
        response = self.tokenizer.decode(
            self.chat_history_ids[:, self.bot_input_ids.shape[i]:][0], skip_special_tokens=True
        )

        while not response:
            i -= 1
            response = self.tokenizer.decode(
                self.chat_history_ids[:, self.bot_input_ids.shape[i]:][0], skip_special_tokens=True
            )


        if response.strip() == '?':
            return "I don't know"
        else:
            return "Okay"

In [ ]:
bot = ChatBot()

while True:
    bot.user_input()
    if bot.end_chat:
        break
    bot.generate_response()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

ChatBot : Welcome, I am a ChatBot, ready to assist you.
 Type 'bye', 'quit', or 'exit' to end the chat

User    : hello


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


ChatBot : Hiya
User    : what's up
ChatBot : Not much, you?
User    : i don't feel good
ChatBot : I'm sorry
User    : give me an advice
ChatBot : I'm sorry
User    : someone broke my herat
ChatBot : I'm sorry
User    : my name is a
ChatBot : I'm sorry


KeyboardInterrupt: Interrupted by user

In [ ]:
import json

# Dosya yolları
input_file = 'new_intents.json'
output_file = 'gpt2_ready_dataset.txt'

# JSON dosyasını yükleyin
with open(input_file, 'r') as file:
    data = json.load(file)

# Eğitim için düz metin formatına dönüştürme
with open(output_file, 'w') as file:
    for intent in data['intents']:
        for pattern in intent['patterns']:
            for response in intent['responses']:
                file.write(f"User: {pattern}\n")
                file.write(f"Bot: {response}\n\n")

print("Dönüşüm tamamlandı ve dosya kaydedildi!")


Dönüşüm tamamlandı ve dosya kaydedildi!


In [ ]:
!pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, TextDataset, DataCollatorForLanguageModeling

# Model ve Tokenizer'ı yükle
model_name = "gpt2"  # Alternatif olarak "gpt2-large" kullanabilirsiniz
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Veri setini yükle
def load_dataset(file_path, tokenizer, block_size=128):
    dataset = TextDataset(
        tokenizer=tokenizer,
        file_path=file_path,
        block_size=block_size
    )
    return dataset

dataset = load_dataset("gpt2_ready_dataset.txt", tokenizer)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [ ]:
# Data collator (maskesiz metin modeli için)
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

# Eğitim parametrelerini ayarla
training_args = TrainingArguments(
    output_dir="./gpt2-finetuned",
    overwrite_output_dir=True,
    num_train_epochs=5,
    per_device_train_batch_size=2,
    save_steps=1000,
    save_total_limit=2,
    prediction_loss_only=True,
    report_to="none"
)

# Trainer'ı başlat
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    data_collator=data_collator,
)

# Eğitimi başlat
trainer.train()

# Modeli kaydet
trainer.save_model("./gpt2-finetuned")
tokenizer.save_pretrained("./gpt2-finetuned")


Step,Training Loss
500,1.415600
1000,0.934000


In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Model ve Tokenizer yükleme
model_name = "./gpt2-finetuned"  # Model dosyanızın yolu
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Chatbot fonksiyonu
def chatbot(user_input):
    # Girdiyi formatla
    input_text = f"User: {user_input}\nBot:"
    input_ids = tokenizer.encode(input_text, return_tensors="pt")

    # Modelle yanıt oluştur
    output = model.generate(
        input_ids,
        max_length=100,
        num_return_sequences=1,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,  # Örnekleme aktif
        temperature=0.7,  # Daha yaratıcı yanıtlar
        top_k=50,         # Çeşitli kelime seçeneklerini sınırlar
        top_p=0.95        # Toplam olasılığa göre kesim yapar
    )

    # Yanıtı decode ederek temizle
    response = tokenizer.decode(output[0], skip_special_tokens=True)

    # Bot'un yanıtını ayıkla
    if "Bot:" in response:
        bot_response = response.split("Bot:")[-1].strip()
    else:
        bot_response = response.strip()

    return bot_response

# Sohbet test
while True:
    user_input = input("You: ")
    if user_input.lower() == "exit":
        print("Exiting chat. Goodbye!")
        break
    print(f"Bot: {chatbot(user_input)}")


Bot: 
Bot: Great to see you. How do you feel currently


KeyboardInterrupt: Interrupted by user